# Invesitgating Bug
## First Recorded 3/7/2017



Version: 2016030021...

In [3]:
"""
General import statements
"""
import crash_analysis
import pandas as pd
from matplotlib import pyplot as plt 
%matplotlib inline

In [11]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReportsIon\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [21]:
"""
Save/load dataframe to/from CSV
"""
from crash_analysis import read_csv

new_data = False
cache_location = 'src/data/IonInvestigation_Mar7_530pm.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = read_csv(cache_location)

In [26]:
"""
Get metadata about dataframe
"""
from crash_analysis import filter_dataframe
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)
# Feb 15 num crashes: 4134

df_dec15 = filter_dataframe(df, AppVersion=versions.keys()[0])
df_nov16 = filter_dataframe(df, AppVersion=versions.keys()[1])

print(col_names)
print(versions)
print(num_crashes)

['Unnamed: 0', 'ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin15.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'InvParamExpression', 'InvParamFile', 'InvParamFunction', 'InvParamLine', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin15.INI', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml', 'empty']
2.015100e+09    1811
2.016030e+09     909
Name: AppVersion, dtype: int64


In [27]:
from crash_analysis import filter_dataframe
# df = filter_dataframe(df, AppVersion=2016070012)
# len(df)
len(df_nov16)

909

In [8]:
"""
Customer Description Analysis
"""
from crash_analysis.analysis import remove_empty, get_column, stem_frequency
customer_desc = remove_empty(get_column(df, 'ProblemDescription'))

stem_frequency(customer_desc)

print

In [30]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe

df = df_nov16

def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]


def print_parings(n, field1='Message', groupings=['AppName', 'InstallType', 'OperatingSystem'], print_output=True):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})
    df_filtered = df_filtered.copy()
    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()

    if print_output:
        print("Query {0} \t count: {1} \t {2}: {3}".format(n+1, count, field1, top_key.encode('ascii', 'ignore')))
        print(field2_hist)
        print(sum(field2_hist))
        print '\n'
    
#

In [68]:
df['ExceptionAddress'].str.endswith('ed0a').value_counts()

def parse_exception_address(exad_str):
    exad_str = exad_str[::-1] # reverse string

    retstr = ''

    if(exad_str[:4] == 'a0de'):
        retstr += 'A'


    else:
        retstr += 'B'

    return retstr

ucrt_df = filter_dataframe(df, ExceptionModule='C:\Windows\System32\ucrtbase.dll')
rev_excep_addr_df = ucrt_df['ExceptionAddress'].apply(parse_exception_address)
# rev_excep_addr_df.srt.startswith('')
rev_excep_addr_df.value_counts()
# print(len(ucrt_df))

height has been deprecated.

Query 1 	 count: 811 	 ExceptionModule: C:\Windows\System32\ucrtbase.dll
CustNum     CrashGUID                             ProblemDescription                                                                                                
0000008012  39f728a9-b5ac-45f8-b736-7e46c92d0ab2  Saving a new client file                                                                                              1
0000132119  0825ec0a-1d82-44e7-bade-e16d140a1e9c  Crash#6 today.  was saving a tax file                                                                                 1
            19e60d34-9d25-4167-bc9d-e2dc561bd428  CRASH #8 TODAY.  WAS ENTERING INFORMATION INTO FORM                                                                   1
            1ba3271a-9a4d-4b42-89ff-d33be47f82d1  Crash #1.  Was saving a tax file                                                                                      1
            30b98404-a954-400e-b58b-1c711db5e6e6  sav

[None]

In [32]:
# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.height', 5000)
# [print_parings(n, groupings='ProblemDescription') for n in range(5)]
# print_parings(0, groupings='CustNum')
# [print_parings(n, field1='ExceptionAddress') for n in [0]]
# [print_parings(n, field1='ExceptionAddress', groupings='ProblemDescription') for n in [0]]

[print_parings(n, field1='ExceptionModule', groupings='ExceptionModuleVersion') for n in range(9)]

# file_counts_df = filter_dataframe(df, ExceptionAddress='0x100427a3')

# file_counts_df = filtered_df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
# file_counts_df.hist(bins=50)
# # file_counts_df = pd.to_numeric(file_counts_df, errors='ignore')
# # file_counts_df.mean()
# print(file_counts_df.min(), file_counts_df.max())
# df[df['CustNum'] == '0614456913']

Query 1 	 count: 235 	 ExceptionModule: C:\Windows\System32\KERNELBASE.dll
6.3.9600.18340    88
10.0.14393.479    60
10.0.14393.0      19
10.0.14393.321    15
6.1.7601.23392    15
6.2.9200.21815    10
6.3.9600.18202     8
10.0.10586.672     4
6.3.9600.17415     3
10.0.10586.589     3
6.3.9600.18007     2
6.3.9600.18264     2
10.0.10586.306     2
6.1.7601.17514     2
6.3.9600.17278     1
10.0.10586.494     1
Name: ExceptionModuleVersion, dtype: int64
235


Query 2 	 count: 230 	 ExceptionModule: C:\ProWin16\32bit\protax16.exe
2016.3.0.21    230
Name: ExceptionModuleVersion, dtype: int64
230


Query 3 	 count: 174 	 ExceptionModule: C:\Windows\SysWOW64\KERNELBASE.dll
6.1.7601.23572    102
6.1.7601.23569     35
6.1.7601.23418     10
6.1.7601.18933      6
6.1.7601.18409      4
6.1.7601.18869      4
6.1.7601.19160      3
6.1.7601.23455      2
6.1.7601.17514      2
6.1.7601.23392      2
6.1.7601.17932      1
6.1.7601.17965      1
6.1.7601.19135      1
6.1.7601.23543      1
Name: ExceptionMod

[None, None, None, None, None, None, None, None, None]

In [24]:
len(df_nov16)

0

In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception